## Overview
My goal is to reproduce the results from the Coetzer and Haines paper for constrained optimization using the coordinate exchange. To do this I will need to compute the region moments matrix and volume of the feasible region with respect to the model. I will also need to compute the verties of the polytope defined by the constraints. Finally, I will need to modify my CEXCH implementation to account for the constrained region. In order to do this, I need to add a function for fetching the candidate directions. For the unconstrained simplex, this should return the Cox direction. For the constrained simplex, the choice is not so obvious. One approach is to just continue using the Cox direction, but compute where the line intersects a constraint, and use that as the vertex. Another option would be to compute the gradient of the function with respect to the current design point.  

## Packages

In [2]:
using Combinatorics
using LinearAlgebra
using IterTools
using IterTools: product

include("../model_builder/model_builder.jl")
using .ModelBuilder

include("../model_builder/design_initializer.jl")
using .DesignInitializer

include("../optimization/optimality_criterion.jl")
using .OptimalityCriterion

include("../utility/tensor_ops.jl")
import .TensorOps: squeeze

using HDF5 
using Optim
using ForwardDiff

using Polyhedra
using CDDLib
using Statistics
using Distributions

## Modified CEXCH

In [3]:
function optimize_point(X, row, d, obj_crit)
    og_row = copy(X[row, :])
    function univariate_obj(x)
        X[row, :] .= og_row .+ x * d
        score = obj_crit(X)
        X[row, :] .= og_row
        return score
    end
    return univariate_obj
end

function jl_opt(X, row, d, obj_crit)
    univariate_obj = optimize_point(X, row, d, obj_crit)
    result = Optim.optimize(univariate_obj, 0, 1.0)
    optim_point = X[row, :] .+ Optim.minimizer(result) * d
    return optim_point, Optim.minimum(result)
end

jl_opt (generic function with 1 method)

In [4]:
function get_simplex_vertices(X, row, col, obj_crit)
    verts = I(size(X, 2))
    return [verts[:, col]]
end

function get_constraint_vertices(A, b; affines=BitSet([]))
    p = polyhedron(hrep(A, b, affines), CDDLib.Library())
    verts = vrep(p)
    return collect(points(verts))
end

function constraint_vertex_getter(A, b; affines=BitSet([]))
    verts = get_constraint_vertices(A, b, affines)
    function get_vertices(X, row, col, obj_crit)
        # Find the vertex with the maximum value in the ith position corresponding with the current col
        vert_sim = vert -> dot(vert, X[row, :])
        max_vert_index = argmax(map(vert_sim, verts))
        return [verts[max_vert_index]]
    end
    
    return get_vertices
end

constraint_vertex_getter (generic function with 1 method)

In [22]:
function cexch_optimize(
    X::Matrix{Float64}, 
    obj_crit, 
    get_cand_verts, 
    optimize_design_point; 
    max_iters=1000)

    N, K = size(X)
    # Initialize objective value
    best_score = obj_crit(X)

    # Iterate until no improvement is made
    for iter in 1:max_iters
        improvement = false

        for coord in CartesianIndices(X)
            row, col = coord[1], coord[2]

            # Get candidate vertices
            vs = get_cand_verts(X, row, col, obj_crit)

            best_vert_score = Inf
            score_opt = Inf
            best_vert = zeros(K)

            # Iterate over candidate vertices
            for v in vs
                # Get the direction vector
                d = v - X[row, :]
                
                # Generate candidate design point optimizing along direction
                (opt_design_point, score_opt) = optimize_design_point(X, row, d, obj_crit)

                # Update the best vertex if improvement is found
                if score_opt < best_vert_score
                    best_vert_score = score_opt
                    best_vert = opt_design_point
                end
            end

            # Update the design matrix and objective value if improvement is found
            if score_opt < best_score
                best_score = score_opt
                X[row, :] .= best_vert
                improvement = true
            end
        end

        if !improvement
            break
        end
    end

    return X
end

cexch_constrained! (generic function with 1 method)

## Testing

In [5]:
N, K = 12, 3
model_builder = ModelBuilder.full_cubic
obj_crit = OptimalityCriterion.d_criterion ∘ model_builder

A = [
    0 -1 0;
    0 0 1;
    5 4 0;
    -20 5 0;
]

b = [
    -1/10;
    3/5;
    39/10;
    -3;
]

4-element Vector{Float64}:
 -0.1
  0.6
  3.9
 -3.0

In [ ]:
init = DesignInitializer.initializer(N, K, model_builder, type="mixture")
designs = init(10_000)

verts = get_constraint_vertices(A, b, BitSet([4]))
m_vert = mean(verts, dims=1)[1]
design = repeat(m_vert', 12)
vert_getter = (X, row, col, obj_crit) -> verts
opt_design = cexch_optimize(design, obj_crit, vert_getter, jl_opt)

## Sanity Check

In [ ]:
model_builder = ModelBuilder.scheffe(2)
init = DesignInitializer.initializer(6, 3, model_builder, type="mixture")
obj_crit = OptimalityCriterion.d_criterion_2 ∘ model_builder
designs = init(10)

In [ ]:
opt_design = cexch_optimize(designs[1, :, :], obj_crit, get_simplex_vertices, jl_opt)

In [ ]:
m = model_builder(opt_design)
OptimalityCriterion.d_criterion_2(m)